                   Marie Diane  Iradukunda
                   Efiche assessigment codes part 

## **Connect to a PostgreSQL Database**


This code attempts to connect to a PostgreSQL database.  
If the connection is successful, it prints a success message.  
If the connection fails, it displays the error message.


In [94]:
import psycopg2

try:
    conn = psycopg2.connect(
        dbname="Efiche_assessment1",
        user="postgres",
        password="didi20189@",
        host="localhost",
        port="5432"
    )
    print("Connected successfully!")
    conn.close()
except Exception as e:
    print(" Connection failed:", e)


Connected successfully!


install the faker module 

In [3]:
!pip install faker


**This code generates 5000 synthetic patient records using the Faker library.**  
**It connects to a PostgreSQL database and inserts each fake patient into the `patients` table.**  
If a patient ID already exists, the record is skipped to avoid conflicts.


In [96]:
import psycopg2
import random
from faker import Faker

fake = Faker()

conn = psycopg2.connect(
    dbname="Efiche_assessment1",
    user="postgres",
    password="didi20189@",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

print("Generating 5000 synthetic patients...")

for _ in range(5000):
    patient_id = fake.uuid4().replace("-", "")[:12]
    age = random.randint(1, 95)
    sex = random.choice(['M', 'F'])
    location = f"{fake.city()}, {fake.state_abbr()}"

    cur.execute("""
        INSERT INTO patients (patient_id, age, sex, location)
        VALUES (%s, %s, %s, %s)
        ON CONFLICT (patient_id) DO NOTHING
    """, (patient_id, age, sex, location))

conn.commit()
cur.close()
conn.close()
print("5000 synthetic patients inserted successfully!")

Generating 5000 synthetic patients...
5000 synthetic patients inserted successfully!


**This script builds an automated data ingestion pipeline for the PadChest dataset.**  
**It reads the CSV file in chunks, cleans the data, and inserts encounters, reports, procedures, and diagnoses into the PostgreSQL database.**  
It also tracks progress using a state file so the pipeline can pause, restart, and continue until all 10,000 rows are processed.


In [128]:

"""
eFiche Assessment – Part 2: Continuous PadChest Ingestion Pipeline
- Incremental (stateful) ingestion of first 10,000+ rows
- Handles duplicates gracefully
- Random patient matching (as per requirement #4)
- Works with official Part 1 schema (TEXT IDs)
"""

import os, json, time, random, pandas as pd, psycopg2
from psycopg2.extras import execute_values

DB_CONFIG = {"dbname": "Efiche_assessment1", "user": "postgres", "password": "didi20189@", "host": "localhost", "port": "5432"}
CSV_PATH = "C:/Users/M.Diane/Desktop/padchest_metadata.csv"  
STATE_FILE = "pipeline_state.json"
CHUNK_SIZE = 5000
SLEEP_SECONDS = 60
TARGET_ROWS = 10000

def load_state():
    return json.load(open(STATE_FILE)).get("processed_rows", 0) if os.path.exists(STATE_FILE) else 0

def save_state(rows):
    json.dump({"processed_rows": rows}, open(STATE_FILE, "w"))

def read_new_rows(start_row):
    if not os.path.exists(CSV_PATH):
        print(f"ERROR: CSV file not found at: {CSV_PATH}")
        return pd.DataFrame()
    print(f"Opening CSV: {CSV_PATH}")
    try:
        df = pd.read_csv(CSV_PATH, skiprows=range(1, start_row + 1), nrows=CHUNK_SIZE, dtype=str, on_bad_lines="skip", encoding="utf-8")
        print(f"Read {len(df)} new rows (from row {start_row})")
        return df
    except Exception as e:
        print(f"CSV read error: {e}")
        return pd.DataFrame()

def ingest_chunk(df):
    if df.empty: return 0
    conn = psycopg2.connect(**DB_CONFIG); cur = conn.cursor()
    cur.execute("SELECT patient_id FROM patients"); patients = [r[0] for r in cur.fetchall()]
    if not patients:
        print("No patients in DB — run synthetic data first!"); conn.close(); return 0

    # Encounters + Reports
    data = [(str(r["StudyID"]), random.choice(patients), "1",
             pd.to_datetime(r["StudyDate_DICOM"], format="%Y%m%d", errors="coerce").date(),
             str(r["StudyID"]), str(r["StudyID"]), r.get("Report") or None, "es") for _, r in df.iterrows()]

    inserted_enc = updated_rep = 0
    sql = """
        INSERT INTO encounters VALUES (%s,%s,%s,%s) ON CONFLICT DO NOTHING;
        INSERT INTO reports VALUES (%s,%s,%s,%s)
        ON CONFLICT (report_id) DO UPDATE SET text = EXCLUDED.text
        RETURNING (xmax = 0);
    """
    for rec in data:
        cur.execute(sql, rec)
        if cur.rowcount: inserted_enc += 1
        if cur.fetchone() and not cur.fetchone()[0]: updated_rep += 1

    # Procedures
    proc = [(str(r["ImageID"]), str(r["StudyID"]), "Chest X-Ray", r.get("Modality_DICOM"), r.get("Projection")) for _, r in df.iterrows()]
    inserted_proc = 0
    if proc:
        execute_values(cur, "INSERT INTO procedures VALUES %s ON CONFLICT DO NOTHING RETURNING procedure_id", proc)
        inserted_proc = cur.rowcount

    # Diagnoses + Junction
    diag_codes = set(); junc = []
    if "Labels" in df.columns:
        for _, r in df.iterrows():
            labels = r["Labels"]; sid = str(r["StudyID"])
            if pd.isna(labels) or str(labels).strip() in ("[]", "nan", ""): continue
            codes = [c.strip().strip("'") for c in str(labels).strip("[]").split(",") if c.strip()]
            for c in codes:
                diag_codes.add(c); junc.append((sid, c))
    inserted_diag = 0
    if diag_codes:
        execute_values(cur, "INSERT INTO diagnoses (code,description) VALUES %s ON CONFLICT DO NOTHING RETURNING code", [(c,c) for c in diag_codes])
        inserted_diag = cur.rowcount
    inserted_junc = 0
    for enc_id, code in junc:
        cur.execute("INSERT INTO encounter_diagnoses (encounter_id,diagnosis_id) SELECT %s,diagnosis_id FROM diagnoses WHERE code=%s ON CONFLICT DO NOTHING RETURNING encounter_id", (enc_id, code))
        if cur.rowcount: inserted_junc += 1

    conn.commit(); cur.close(); conn.close()

    print(f"Inserted/updated ~{len(df)}+ records (encounters, procedures, diagnoses)")
    print(f"   -> Encounters: {inserted_enc} inserted, {len(data)-inserted_enc} duplicates skipped")
    print(f"   -> Reports: {updated_rep} text fields updated")
    print(f"   -> Procedures: {inserted_proc} inserted, {len(proc)-inserted_proc} duplicates skipped")
    print(f"   -> Diagnoses: {inserted_diag} new codes added")
    print(f"   -> Encounter-Diagnoses: {inserted_junc} links created")
    return len(df)

if __name__ == "__main__":
    print("eFiche PadChest Ingestion Pipeline STARTED")
    processed = load_state()
    while processed < TARGET_ROWS:
        df = read_new_rows(processed)
        if df.empty:
            print("No more data in CSV. Pipeline complete.")
            break
        processed += ingest_chunk(df)
        save_state(processed)
        print(f"Total processed: {processed}/{TARGET_ROWS} rows")
        if processed < TARGET_ROWS:
            print(f"Sleeping {SLEEP_SECONDS} seconds...\n")
            time.sleep(SLEEP_SECONDS)
    print("\n COMPLETED — 10,000+ rows ingested successfully!")

eFiche PadChest Ingestion Pipeline STARTED

 COMPLETED — 10,000+ rows ingested successfully!


Intrepretation :
The script connects to PostgreSQL and fills the date and dimension tables (patients, procedures, diagnoses) from the source system.

It then builds the fact_encounter table by joining encounters with their related patient, procedure, diagnosis, date, and report.

Finally, it commits all inserts and fully populates the data warehouse (DWH).

In [100]:

import psycopg2
from psycopg2.extras import execute_values

conn = psycopg2.connect(
    dbname="Efiche_assessment1",
    user="postgres",
    password="didi20189@",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

print("Populating dim_date (2010–2025)...")
cur.execute("""
    INSERT INTO dwh.dim_date (full_date, year, month, day, quarter, month_name, day_of_week, is_weekend)
    SELECT d::date,
           EXTRACT(YEAR FROM d)::int,
           EXTRACT(MONTH FROM d)::int,
           EXTRACT(DAY FROM d)::int,
           EXTRACT(QUARTER FROM d)::int,
           TO_CHAR(d, 'Month'),
           TO_CHAR(d, 'Day'),
           CASE WHEN EXTRACT(DOW FROM d) IN (0,6) THEN TRUE ELSE FALSE END
    FROM generate_series('2010-01-01'::date, '2025-12-31'::date, '1 day'::interval) d
    ON CONFLICT (full_date) DO NOTHING;
""")

print("ETL: Loading dimensions and fact table...")

# 1. dim_patient
cur.execute("""
    INSERT INTO dwh.dim_patient (patient_id, age, sex, location)
    SELECT patient_id, age, sex, location FROM patients
    ON CONFLICT (patient_id) DO NOTHING;
""")

# 2. dim_procedure
cur.execute("""
    INSERT INTO dwh.dim_procedure (procedure_id, procedure_name, modality, projection)
    SELECT procedure_id, procedure_name, modality, projection FROM procedures
    ON CONFLICT (procedure_id) DO NOTHING;
""")

# 3. dim_diagnosis
cur.execute("""
    INSERT INTO dwh.dim_diagnosis (diagnosis_id, code, description)
    SELECT diagnosis_id, code, description FROM diagnoses
    ON CONFLICT (diagnosis_id) DO NOTHING;
""")

# 4. fact_encounter (NOW WORKS!)
cur.execute("""
    INSERT INTO dwh.fact_encounter (
        encounter_id, patient_key, procedure_key, diagnosis_key, date_key, report_text, report_language
    )
    SELECT 
        e.encounter_id,
        dp.patient_key,
        dpr.procedure_key,
        dd.diagnosis_key,
        ddate.date_key,
        r.text,
        r.language
    FROM encounters e
    JOIN patients p ON e.patient_id = p.patient_id
    JOIN dwh.dim_patient dp ON p.patient_id = dp.patient_id
    JOIN procedures pr ON e.encounter_id = pr.encounter_id
    JOIN dwh.dim_procedure dpr ON pr.procedure_id = dpr.procedure_id
    LEFT JOIN encounter_diagnoses ed ON e.encounter_id = ed.encounter_id
    LEFT JOIN dwh.dim_diagnosis dd ON ed.diagnosis_id = dd.diagnosis_id
    JOIN dwh.dim_date ddate ON e.encounter_date = ddate.full_date
    LEFT JOIN reports r ON e.encounter_id = r.encounter_id
    ON CONFLICT (encounter_id) DO NOTHING;
""")

conn.commit()
print("PART 3 ETL COMPLETE — DWH FULLY POPULATED")
cur.close()
conn.close()

Populating dim_date (2010–2025)...
ETL: Loading dimensions and fact table...
PART 3 ETL COMPLETE — DWH FULLY POPULATED


## DASHBOARD

In [84]:
import plotly.express as px
import pandas as pd
import psycopg2
from dash import Dash, html, dcc
import dash_bootstrap_components as dbc
import os
import base64

# ==============================
# 1. Database Connection
# ==============================
def get_connection():
    try:
        conn = psycopg2.connect(
            dbname="Efiche_assessment1",
            user="postgres",
            password="didi20189@",
            host="localhost",
            port="5432"
        )
        print("Database connected successfully.")
        return conn
    except Exception as e:
        print(f"Database connection failed: {e}")
        exit(1)

conn = get_connection()

# ==============================
# 2. Load Data
# ==============================
try:
    print("Loading data from database...")
    # 1. Encounters per Month
    df_monthly = pd.read_sql("""
        SELECT dd.year, dd.month_name, dd.month, COUNT(*) AS encounters
        FROM dwh.fact_encounter f
        JOIN dwh.dim_date dd ON f.date_key = dd.date_key
        GROUP BY dd.year, dd.month_name, dd.month
        ORDER BY dd.year, dd.month
    """, conn)

    # 2. Top Diagnoses by Age Group
    df_diag = pd.read_sql("""
        WITH age_groups AS (
            SELECT patient_key,
                   CASE WHEN age < 18 THEN '0-17'
                        WHEN age BETWEEN 18 AND 34 THEN '18-34'
                        WHEN age BETWEEN 35 AND 54 THEN '35-54'
                        ELSE '55+' END AS age_group
            FROM dwh.dim_patient WHERE is_current
        )
        SELECT ag.age_group, ddiag.code, COUNT(*) AS count
        FROM dwh.fact_encounter f
        JOIN age_groups ag ON f.patient_key = ag.patient_key
        JOIN dwh.dim_diagnosis ddiag ON f.diagnosis_key = ddiag.diagnosis_key
        GROUP BY ag.age_group, ddiag.code
        ORDER BY count DESC LIMIT 15
    """, conn)

    # 3. Average Studies per Patient
    avg_studies = pd.read_sql("""
        SELECT ROUND(AVG(study_count)::numeric, 2) AS avg_studies_per_patient
        FROM (
            SELECT patient_id, COUNT(*) AS study_count
            FROM encounters
            GROUP BY patient_id
        ) t
    """, conn).iloc[0]['avg_studies_per_patient']

    # 4. Top Diagnosis Clusters
    df_clusters = pd.read_sql("""
        SELECT word, COUNT(*) AS frequency
        FROM (
            SELECT unnest(string_to_array(lower(regexp_replace(COALESCE(text, ''), '[^a-záéíóúñ ]', '', 'g')), ' ')) AS word
            FROM reports
            WHERE text IS NOT NULL AND text <> ''
        ) words
        WHERE word IN ('pneumonia', 'cardiomegaly', 'edema', 'effusion', 'atelectasis',
                       'normal', 'consolidation', 'infiltrates', 'opacity', 'fracture', 'mass', 'nodule')
        GROUP BY word
        ORDER BY frequency DESC
    """, conn)

    print("Data loaded successfully.")
except Exception as e:
    print(f"Query failed: {e}")
    conn.close()
    exit(1)
finally:
    conn.close()

# ==============================
# 3. Create Figures
# ==============================
# Fig 1: Encounters per Month
fig1 = px.bar(
    df_monthly,
    x="month_name",
    y="encounters",
    color="year",
    title="Number of encounters per month ",
    labels={"encounters": "Number of Studies", "month_name": "Month"},
    height=520,
    template="simple_white"
)
fig1.update_layout(barmode='group', legend_title="Year", title_x=0.5, font=dict(family="Arial, sans-serif"))

# Fig 2: Top Diagnoses by Age Group
fig2 = px.bar(
    df_diag,
    x="code",
    y="count",
    color="age_group",
    title="Top 15 Diagnoses by Age Group",
    labels={"count": "Diagnosis Count", "code": "Diagnosis Code"},
    barmode="group",
    height=520,
    template="simple_white"
)
fig2.update_layout(xaxis_tickangle=45, legend_title="Age Group", title_x=0.5)

# Fig 4: Diagnosis Clusters
fig4 = px.bar(
    df_clusters,
    x="word",
    y="frequency",
    color="frequency",
    title="Top Diagnosis Clusters from Report Text",
    labels={"frequency": "Frequency", "word": "Clinical Term"},
    height=500,
    template="simple_white",
    color_continuous_scale="Viridis"
)
fig4.update_layout(xaxis_tickangle=45, title_x=0.5)

# ==============================
# 4. Save Individual HTML Files
# ==============================
def save_individual_html():
    os.makedirs("submission", exist_ok=True)
    fig1.write_html("submission/dashboard_encounters_by_month.html")
    fig2.write_html("submission/dashboard_diagnoses_by_age.html")
    fig4.write_html("submission/dashboard_diagnosis_clusters.html")
    print("3 HTML files saved in /submission folder!")

save_individual_html()

# ==============================
# 5. Generate Full Static HTML + Auto-Download
# ==============================
def generate_full_html():
    html_content = f"""<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1.0"/>
  <title>Clinical Analytics Dashboard</title>
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/css/bootstrap.min.css" rel="stylesheet"/>
  <script src="https://cdn.plot.ly/plotly-2.27.0.min.js"></script>
  <style>
    body {{ background: #f8f9fa; font-family: 'Segoe UI', sans-serif; padding: 20px; }}
    .kpi-card {{ background: linear-gradient(135deg, #3498db, #2980b9); color: white; border-radius: 15px; box-shadow: 0 8px 20px rgba(0,0,0,0.15); }}
    .kpi-value {{ font-size: 3.2rem; font-weight: 800; }}
    .chart-box {{ background: white; border-radius: 15px; padding: 20px; margin-bottom: 25px; box-shadow: 0 4px 15px rgba(0,0,0,0.08); }}
    .footer {{ margin-top: 50px; color: #6c757d; }}
    h1 {{ color: #2c3e50; }}
    .download-btn {{ position: fixed; top: 20px; right: 20px; z-index: 1000; }}
  </style>
</head>
<body>
  <!-- Download Button -->
  <button class="btn btn-success download-btn shadow-lg" onclick="downloadDashboard()">
    Download Dashboard (HTML)
  </button>

  <div class="container-fluid">
    <h1 class="text-center mb-4 fw-bold">Clinical Analytics Dashboard</h1>
    <!-- KPI -->
    <div class="row justify-content-center mb-5">
      <div class="col-md-4 col-sm-6">
        <div class="card kpi-card text-center p-3">
          <div class="card-body">
            <div class="kpi-value">{avg_studies}</div>
            <p class="mb-0 fw-bold">Average Studies per Patient</p>
            <small> </small>
          </div>
        </div>
      </div>
    </div>
    <!-- Charts -->
    <div class="row">
      <div class="col-lg-6"><div class="chart-box" id="chart1"></div></div>
      <div class="col-lg-6"><div class="chart-box" id="chart2"></div></div>
    </div>
    <div class="row justify-content-center">
      <div class="col-lg-8"><div class="chart-box" id="chart4"></div></div>
    </div>
    <div class="footer text-center">
      <hr><p>Data Source: PadChest </p>
    </div>
  </div>

  <script>
    const fig1 = {fig1.to_json()};
    const fig2 = {fig2.to_json()};
    const fig4 = {fig4.to_json()};
    Plotly.newPlot('chart1', fig1.data, fig1.layout, {{responsive: true}});
    Plotly.newPlot('chart2', fig2.data, fig2.layout, {{responsive: true}});
    Plotly.newPlot('chart4', fig4.data, fig4.layout, {{responsive: true}});

    // Auto-download after 1 second (optional — remove if unwanted)
    setTimeout(downloadDashboard, 1000);

    function downloadDashboard() {{
      const content = document.documentElement.outerHTML;
      const blob = new Blob([content], {{ type: 'text/html' }});
      const url = URL.createObjectURL(blob);
      const a = document.createElement('a');
      a.href = url;
      a.download = 'clinical_dashboard_full.html';
      document.body.appendChild(a);
      a.click();
      document.body.removeChild(a);
      URL.revokeObjectURL(url);
    }}
  </script>
</body>
</html>"""

    filename = "clinical_dashboard_full.html"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(html_content)
    print(f"{filename} generated with auto-download!")
    print("   → Open in browser: auto-downloads + has download button")

generate_full_html()

# ==============================
# 6. Dash App Setup + Download Button
# ==============================
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Create download link for full HTML
def get_dashboard_download_link():
    with open("clinical_dashboard_full.html", "r", encoding="utf-8") as f:
        html_string = f.read()
    b64 = base64.b64encode(html_string.encode()).decode()
    return html.A(
        html.Button("Download Full Dashboard (HTML)", className="btn btn-success shadow"),
        href=f"data:text/html;base64,{b64}",
        download="clinical_dashboard_full.html",
        className="position-fixed",
        style={"top": "20px", "right": "20px", "zIndex": 1000}
    )

kpi_card = dbc.Card(
    dbc.CardBody([
        html.H3(f"{avg_studies}", className="text-primary fw-bold mb-1"),
        html.P("Average Studies per Patient", className="mb-0"),
        html.Small(" ", className="text-muted")
    ], className="text-center p-4 shadow"),
    className="border-start border-primary border-5"
)

app.layout = dbc.Container([
    get_dashboard_download_link(),  # Download button in live app
    html.H1("Clinical Analytics Dashboard", className="text-center my-4 text-primary fw-bold"),
    dbc.Row(dbc.Col(kpi_card, md=4), className="justify-content-center mb-4"),
    dbc.Row([
        dbc.Col(dcc.Graph(figure=fig1), lg=6, className="mb-4"),
        dbc.Col(dcc.Graph(figure=fig2), lg=6, className="mb-4"),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(figure=fig4), lg=8, className="mx-auto")
    ]),
    html.Hr(),
    html.P("Data Source: PadChest ", className="text-center text-muted small")
], fluid=True, className="py-3")

# ==============================
# 7. Run Server
# ==============================
if __name__ == '__main__':
    print("\n" + "="*60)
    print("BONUS DASHBOARD COMPLETE")
    print("="*60)
    print("1. Live Dashboard → http://127.0.0.1:8050")
    print("2. Static HTML → clinical_dashboard_full.html (auto-downloads on open)")
    print("3. Submission → /submission/*.html")
    print("4. Download buttons in both static & live versions!")
    print("="*60)
   
    app.run(debug=False, port=8050)

Database connected successfully.
Loading data from database...
Data loaded successfully.
3 HTML files saved in /submission folder!
clinical_dashboard_full.html generated with auto-download!
   → Open in browser: auto-downloads + has download button

BONUS DASHBOARD COMPLETE
1. Live Dashboard → http://127.0.0.1:8050
2. Static HTML → clinical_dashboard_full.html (auto-downloads on open)
3. Submission → /submission/*.html
4. Download buttons in both static & live versions!
